## 데이터 전처리 2에서 진행했던, 방언 번역 데이터셋을 KoAlpaca(polyglot-ko)입력에 맞는 프롬프트로 제작한다.

instruction, input, output의 구조를 띠고 있기 때문에, 여기에 맞게 만들어준다.

In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 8.6 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd '/content/drive/MyDrive/한국어_방언_데이터셋/data'

/content/drive/MyDrive/한국어_방언_데이터셋/data


In [4]:
import json
from datasets import load_dataset

file_path = './dialect_cleaning/'       # 이 부분은 preprocessing2.py를 통해 만든 전처리 json이 든 디렉토리의 경로가 될 것.

# 현재 이 부분은 데이터를 좀 더 줄여서 만들었기 때문에, cut_..._clean.json를 로드합니다.
data_files={
    "train":file_path+'cutoff/cut_train_clean.json',
    "valid":file_path+'cutoff/cut_valid_clean.json',
    "test":file_path+'cutoff/cut_test_clean.json'}
dialect_data = load_dataset("json", data_files=data_files)      # 데이터셋 딕셔너리로 로드

train_set = dialect_data['train']
valid_set = dialect_data['valid']
test_set = dialect_data['test']

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating valid split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

### Alpaca형식으로 학습을 진행하기 위해, 프롬프트를 제작하는 공간

In [5]:
def prompt_setting(file_name, data_set):
  convert_data = []

  # 데이터셋 내의 모든 요소들을 변환할 예정
  for item in data_set:
    convert_item = {
      "instruction": "사투리가 포함된 문장이면 표준어로 변환해주시오.",
      "input": item['source'],
      "output": item['target']
      }
    convert_data.append(convert_item)

  with open(file_path+'prompt/'+file_name, "w", encoding="utf-8") as f:
    json.dump(convert_data, f, ensure_ascii=False, indent=4)

  return

In [6]:
prompt_setting("방언번역_cut_train_set_prompt.json", train_set)
prompt_setting("방언번역_cut_valid_set_prompt.json", valid_set)
prompt_setting("방언번역_cut_test_set_prompt.json", test_set)

#### 프롬프트 탬플릿을 만들어주는 공간. Alpaca의 형식과 조금 달라지기에 Custom 탬플릿을 제작함

In [ ]:
# GPT, LLama, Alpaca 등 LLM 모델에 필요한 template 파일을 정의하는 코드입니다.
# 아래 코드는 Alpaca-Lora가 인식하는 template의 형식입니다. 아래처럼 진행해서, 마찬가지로 json 파일로 저장해주시면 됩니다.

prompt_template = {
    "description" : "Polyglot-ko Alpaca-QLora Custom Template",
    "prompt_input" : (
        "Below is an instruction that describes a task, Paired with an input that provides further context.\n"
        "Writhe a response that appropriately completes the request.\n"
        "### 명령어 : \n{instruction}\n\n### 원문 : \n{input}\n\n### 번역:\n"
    ),
    "prompt_no_input" : (
        "Below is an instruction that describes a task.\n"
        "Writhe a response that appropriately completes the request.\n"
        "### 명령어 : \n{instruction}\n\n### 번역:\n"
    ),
    "response_split" : "### 번역:"
}

with open("/content/drive/MyDrive/KB 공모전/Polyglot-ko-dialect 만들기/Alpaca-QLoRA/templates/custom_template.json", "w", encoding='utf-8') as f:
    json.dump(prompt_template, f, ensure_ascii=False)